In [1]:
#defines

base_path = 'C:/Users/I078212/workspace/plc/hana/'

xsprivileges_files = [
    base_path + 'src/PLC/sap/plc_repl/xs/.xsprivileges',
    base_path + 'src/PLC/sap/plc/doc/.xsprivileges',
    base_path + 'src/PLC/sap/plc/xs/.xsprivileges'
]

print(xsprivileges_files)


''' src/PLC/sap/plc/doc/.xsprivileges as an example:
{
    "privileges" : [
        { "name" : "API_Documentation_Read", "description": "Read access for the swagger api documentation"}
    ]
}
'''

['C:/Users/I078212/workspace/plc/hana/src/PLC/sap/plc_repl/xs/.xsprivileges', 'C:/Users/I078212/workspace/plc/hana/src/PLC/sap/plc/doc/.xsprivileges', 'C:/Users/I078212/workspace/plc/hana/src/PLC/sap/plc/xs/.xsprivileges']


' src/PLC/sap/plc/doc/.xsprivileges as an example:\n{\n    "privileges" : [\n        { "name" : "API_Documentation_Read", "description": "Read access for the swagger api documentation"}\n    ]\n}\n'

In [2]:
# parse .xsprivileges which are in json files

import json

in_jsons = []

for f in xsprivileges_files:
    with open(f) as json_data:
        in_jsons.append(json.load(json_data))

print(in_jsons[0])

{'privileges': [{'name': 'generators', 'description': 'Allows usage of artifact generators.'}]}


In [3]:
# generate scopes, role-templates

scopes = []
for data in in_jsons:
    for priviledge in data['privileges']:
        scope = {}
        scope['name'] = '$XSAPPNAME.' + priviledge['name']
        scope['description'] = priviledge['description']
        scopes.append(scope)
#print(scopes)

role_templates = []

# add the most basic role templates, each is associated with one scope
for data in in_jsons:
    for priviledge in data['privileges']:
        role_template = {}
        role_template['name'] = priviledge['name'] + '_RT'
        role_template['scope-references'] = ['$XSAPPNAME.' + priviledge['name']]
        role_templates.append(role_template)


In [12]:
# more role-templates
root_path = 'C:/Users/I078212/workspace/plc/hana/src/PLC/sap/'
xsc_roles_files = [
    'plc/authorizations/Addin_Administrator.hdbrole',
    'plc/authorizations/Addin_User.hdbrole',
    'plc/authorizations/Base_Viewer.hdbrole',
    # TODO: and more ...
]

xsc_roles_data = []

# prepare xsc_roles_data[]
for i in range(len(xsc_roles_files)):
    xsc_roles_files[i] = root_path + xsc_roles_files[i]
    with open(xsc_roles_files[i], 'r') as my_file:
        file_data = my_file.read().replace('\n', '')
        xsc_roles_data.append(file_data.replace('\t', ' '))

#print(xsc_roles_files)
print(xsc_roles_data)

''' XSC role file example:
role sap.plc.authorizations::Contributor
    extends role sap.plc.authorizations::Calculation_Version_Viewer, 
    sap.plc.authorizations::Admin_Costing_And_Analysis_Viewer, 
    sap.plc.authorizations::Admin_Finance_Viewer, 
    sap.plc.authorizations::Admin_Global_Settings_Viewer,
    sap.plc.authorizations::Admin_Logistics_Viewer,
    sap.plc.authorizations::Admin_Prices_Viewer	
{
    application privilege: sap.plc.xs::Projects_Create_Update;
    application privilege: sap.plc.xs::Calculations_Create_Update;
    application privilege: sap.plc.xs::Calculation_Versions_Create_Update;
    application privilege: sap.plc.xs::Calculation_Versions_Freeze;
    application privilege: sap.plc.xs::Users_Read;
}
'''
def parse_hdbrole(role_file):
    parsed_role = {}
    # TODO:
    return parsed_role



['role sap.plc.authorizations::Addin_Administrator extends role sap.plc.authorizations::Addin_User{application privilege: "sap.plc.xs::AddIn_Edit";application privilege: "sap.plc.xs::AddIn_Read_All";}', 'role sap.plc.authorizations::Addin_User extends role sap.plc.authorizations::Base_Viewer{application privilege: "sap.plc.xs::AddIn_Read_Activated";}', 'role sap.plc.authorizations::Base_Viewer{ application privilege: "sap.plc.xs::Base_Access";  application privilege: "sap.plc.xs::User_Default_Settings_Edit"; application privilege: "sap.plc.xs::Tasks_Read";}']


In [4]:
# write output file
xs_security_file = 'C:/Users/I078212/tmp/xs-security.json'

xs_security = {}
xs_security['xsappname'] = 'HCO_PLC'
xs_security['scopes'] = scopes
xs_security['role-templates'] = role_templates
#print(xs_security)

import json
with open(xs_security_file, 'w') as out_file:
    out_file.write(json.dumps(xs_security, ensure_ascii=True, indent=4))
